<a href="https://colab.research.google.com/github/xiaoyufan/speech-data-augmentation/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

### Create logger

In [1]:
from importlib import reload
import logging
import sys

reload(logging)

LOGGING_LEVEL = 'DEBUG'

logger = logging.getLogger('baseline')
logger.setLevel(LOGGING_LEVEL)

formatter = logging.Formatter('[%(asctime)s - logger %(name)s - %(levelname)s] %(message)s')

ch = logging.StreamHandler(sys.stdout)
ch.setFormatter(formatter)
logger.addHandler(ch)

logger.debug('debug test')
logger.info('info test')

[2020-12-15 17:36:29,678 - logger baseline - DEBUG] debug test
[2020-12-15 17:36:29,680 - logger baseline - INFO] info test


### Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Configurations

#### Configure mode

In [3]:
MODES = {
  'BASELINE': 'baseline',
  'TEST': 'test',
}
MODE =  MODES['BASELINE']
logger.info(f'Notebook runs in {MODE} mode.')

[2020-12-15 17:38:37,878 - logger baseline - INFO] Notebook runs in baseline mode.


#### Get notebook's start time

In [4]:
from datetime import datetime, tzinfo
import pytz

NB_RUN_TIME = datetime.now(tz=pytz.timezone('US/Eastern')).strftime('%Y%m%d-%H%M%S')
logger.info(f'Notebook started at {NB_RUN_TIME}.')

[2020-12-15 17:38:37,917 - logger baseline - INFO] Notebook started at 20201215-123837.


#### Other configurations

In [5]:
DEEPSPEECH_LOG_LEVEL = '1'
DEEPSPEECH_PATH = '/content/DeepSpeech'
PROJECT_ROOT_PATH = '/content/drive/MyDrive/nlp-project'

DATASET_PATH = f'{PROJECT_ROOT_PATH}/cmu_arctic'
WAV_TEST_DIR = f'{DATASET_PATH}/test/audio'

OUTPUT_PATH = f'{PROJECT_ROOT_PATH}/xiaoyu-baseline/20201215-113234'
MODEL_PATH = f'{OUTPUT_PATH}/models/output_graph.pb'

### Install packages

In [6]:
%%bash -s "$DEEPSPEECH_PATH"
DEEPSPEECH_PATH=$1

if [ ! -d "$DEEPSPEECH_PATH" ] ; then
  git clone --branch v0.9.2 https://github.com/mozilla/DeepSpeech $DEEPSPEECH_PATH
fi

cd $DEEPSPEECH_PATH
pip install --upgrade pip==20.2.2 wheel==0.34.2 setuptools==49.6.0
pip install --upgrade -e .

# pip uninstall tensorflow -y
pip install --upgrade tensorflow==1.15.4
pip install tensorflow-gpu==1.15.4

# Install other packages
pip install pandas
# tensorflow 1.15.4 requires numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.4 which is incompatible.
pip install --upgrade numpy==1.16.0

  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
  Found existing installation: wheel 0.36.1
    Uninstalling wheel-0.36.1:
      Successfully uninstalled wheel-0.36.1
  Found existing installation: setuptools 50.3.2
    Uninstalling setuptools-50.3.2:
      Successfully uninstalled setuptools-50.3.2
Obtaining file:///content/DeepSpeech
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for opuslib: filename=opuslib-2.0.0-py3-none-any.whl size=11009 sha256=beff30d4a10c8e63861c805e2672131bf44fe79f5c68bdf7dc79d0ed60cffe2c
  Stored in directory: /root/.cache/pip/wheels/6c/01/88/37797e9e9d157a33eefed22a46aa0bf5044effcec6a9181e41
  Created wheel for optuna

Cloning into '/content/DeepSpeech'...
Note: checking out 'b2920c755717499fad8e49f2c27091f438357653'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

umap-learn 0.4.6 requires numba!=0.47,>=0.46, but you'll have numba 0.47.0 which is incompa

In [7]:
%%bash

pip install --upgrade deepspeech-gpu==0.9.2

#  Inference

In [9]:
%%bash -s "$MODEL_PATH" "$WAV_TEST_DIR"
MODEL_PATH=$1
WAV_TEST_DIR=$2

deepspeech --model $MODEL_PATH --audio $WAV_TEST_DIR/female_40/

2020-12-15 17:45:33.352089: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
TensorFlow: v2.3.0-6-g23ad988
DeepSpeech: v0.9.2-0-gb2920c7
2020-12-15 17:45:33.477293: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-12-15 17:45:33.479038: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-12-15 17:45:33.508882: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-15 17:45:33.509555: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716]